## Part 2  -  VacationPy - Choose Vacation Spots with Ideal Weather

### Introduction 

In VacationPy, I have used the hvplot.pandas plotting library to plot all the cities on a world map. After that, I began searching for a list of cities experiencing excellent weather that would suit a summer holiday. I copied my new dataframe, 'city_data_vac_df', from 'city_data_time_df' restricting 'Max Temp to between 22 and 30, Wind Speed to less than 4.5 m/s and cloudiness less than 5%. I found 13 cities that met my requirements. I used the Geoapify API to find a hotel less than 10 km from the city centre for each city. I printed a map of the cities and added the hotel's name and country to hover information.

Note that both the World Map with all data points, and the Vacation Spot Map take some time generate and plot.

### Load and Set Up Dependcies

#### Using hvplot. pandas to plot maps
##### I also loaded cartopy, geoviews and pyproj libraries using pip install Jupiter Notebook.

In [1]:
# Dependencies and Setup

# !pip install cartopy
# !pip install geoviews
# !pip install puproj

import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/citiestime.csv")
city_data_time_df = pd.read_csv(csv_file)

# Display sample data
city_data_time_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
0,0,Berdigestyakh,62.0989,126.6992,-24.72,97,100,2.32,RU,2024-01-01,02:49:36
1,1,Tiksi,71.6872,128.8694,-29.57,99,100,1.01,RU,2024-01-01,02:49:37
2,2,Waitangi,-43.9535,-176.5597,16.01,84,100,2.24,NZ,2024-01-01,02:49:37
3,3,Tura,25.5198,90.2201,20.29,51,0,0.55,IN,2024-01-01,02:49:38
4,4,Bilibino,68.0546,166.4372,-30.68,87,88,2.26,RU,2024-01-01,02:49:38


### Creating a map that displays a point for every city in 'city_data_time_df' DataFrame.

#### The size and colour of each point is dependent on the humidity of each location.

In [3]:
# configure the map plot.

# Ensure hvplot is enabled for geographic plotting
hvplot.extension('bokeh')

# Get data acquisition date.
fetch_date = city_data_time_df['Date'][0]

# Configuring the map plot
map_plot = city_data_time_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,  # Enable geographic coordinates
    size='Humidity',  # Use 'Humidity' for point sizes
    hover_cols=['City', 'Country', 'Humidity'],  # Data to show on hover
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=1000,
    frame_height=600,
    color='Humidity',  # Color points based on 'Humidity'
    cmap='inferno',  # Color map
    alpha=0.9,  # Transparency of the points
    xlim=(-180, +180),  # Set the longitude range
    ylim=(-50, +70),     # Set the latitude range
    xlabel='Longitude',  # Label for the X-axis
    ylabel='Latitude',   # Label for the Y-axis
    title=f'City Locations and Humidity  -  {fetch_date}'  # Title of the plot
)
    # display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Narrow down cities in "city_data_time_df" to find a place with ideal weather

 #### Ideal weather deemed to be 22 <= Max Temp <= 30, Wind Speed < 4.5, and Cloudiness = 0

#### Ideal holiday weather deemed to be 22<=Max Temp <=30, Wind Speed <4.5 m/s , Cloudiness <=5%

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_vac_df = city_data_time_df[
    (city_data_time_df['Max Temp'] >= 22) & 
    (city_data_time_df['Max Temp'] <= 30) &
    (city_data_time_df['Wind Speed'] < 4.5) &
    (city_data_time_df['Cloudiness'] <= 5
    )
]

# Drop any rows with null values
city_data_vac_df = city_data_vac_df.dropna()

# Display sample data

city_data_vac_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
74,74,Sarandi,-23.4436,-51.8739,25.75,53,0,4.12,BR,2024-01-01,02:50:07
150,150,Mar Del Plata,-38.0023,-57.5575,22.19,66,0,3.58,AR,2024-01-01,02:50:42
179,179,Baardheere,2.3446,42.2764,24.89,69,1,2.31,SO,2024-01-01,02:50:54
185,185,Lazaro Cardenas,17.9583,-102.2000,23.08,76,5,1.70,MX,2024-01-01,02:50:56
196,196,Twon-Brass,4.3151,6.2418,27.24,79,3,3.26,NG,2024-01-01,02:51:02


### Create a new DataFrame called Hotel_df, and add an extra column called 'Hotel Name'

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_vac_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
74,Sarandi,BR,-23.4436,-51.8739,53,NaN
150,Mar Del Plata,AR,-38.0023,-57.5575,66,NaN
179,Baardheere,SO,2.3446,42.2764,69,NaN
185,Lazaro Cardenas,MX,17.9583,-102.2000,76,NaN
196,Twon-Brass,NG,4.3151,6.2418,79,NaN
257,Puerto San Jose,GT,13.9256,-90.8244,68,NaN
296,Luanda,AO,-8.8368,13.2343,2,NaN
300,Acapulco De Juarez,MX,16.8634,-99.8901,76,NaN
433,La Leonesa,AR,-27.0379,-58.7035,86,NaN
443,Upington,ZA,-28.4478,21.2561,56,NaN


### Find and download close by hotels in each city using Geoapify API.

In [6]:
def find_hotel(lat, lng):
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},10000',
        'limit': 1,
        'apiKey': geoapify_key
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = response.json()
        if data['features']:
            hotel_info = data['features'][0]
            return hotel_info['properties']['name']
        else:
            return "No hotel found"
    except: 
        return f"API request error: {e}"

# Looping through the DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    hotel_name = find_hotel(lat, lng)
    hotel_df.at[index, 'Hotel Name'] = hotel_name

hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
74,Sarandi,BR,-23.4436,-51.8739,53,Hotel Metrópole Maringá
150,Mar Del Plata,AR,-38.0023,-57.5575,66,Hotel Valles. Mar del Plata
179,Baardheere,SO,2.3446,42.2764,69,No hotel found
185,Lazaro Cardenas,MX,17.9583,-102.2000,76,Quinta Antigua Hotel
196,Twon-Brass,NG,4.3151,6.2418,79,KC Hotel
257,Puerto San Jose,GT,13.9256,-90.8244,68,Villa Real
296,Luanda,AO,-8.8368,13.2343,2,Hotel Presidente Luanda
300,Acapulco De Juarez,MX,16.8634,-99.8901,76,Krystal Beach Acapulco
433,La Leonesa,AR,-27.0379,-58.7035,86,Residencial Geminis
443,Upington,ZA,-28.4478,21.2561,56,River City Inn


### For each city, use the Geoapify API to find the first hotel located within 10,000 metres of these coordinates.

In [7]:
def find_hotel(lat, lng):
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},10000',
        'limit': 1,  # we only want the first result
        'apiKey': geoapify_key
    }
   
    #make request and receive response.
    response = requests.get(base_url, params=params)
        
    if response.status_code == 200:
        data = response.json()
                
        if data['features']:
            # Extract hotel information
            hotel_info = data['features'][0]
            hotel_name = hotel_info['properties']['name']
            return hotel_name
        else:
            return "No hotel found"
    else:
        return "API request failed"
    
# Looping through the DataFrame
for index, row in hotel_df.iterrows():
    # Extracting latitude and longitude for each city
    lat = row['Lat']
    lng = row['Lng']
    
    # Calling the find_hotel function
    hotel_name = find_hotel(lat, lng)
    # Storing the result back in the DataFrame
    hotel_df.at[index, 'Hotel Name'] = hotel_name
    
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
74,Sarandi,BR,-23.4436,-51.8739,53,Hotel Metrópole Maringá
150,Mar Del Plata,AR,-38.0023,-57.5575,66,Hotel Valles. Mar del Plata
179,Baardheere,SO,2.3446,42.2764,69,No hotel found
185,Lazaro Cardenas,MX,17.9583,-102.2000,76,Quinta Antigua Hotel
196,Twon-Brass,NG,4.3151,6.2418,79,KC Hotel
257,Puerto San Jose,GT,13.9256,-90.8244,68,Villa Real
296,Luanda,AO,-8.8368,13.2343,2,Hotel Presidente Luanda
300,Acapulco De Juarez,MX,16.8634,-99.8901,76,Krystal Beach Acapulco
433,La Leonesa,AR,-27.0379,-58.7035,86,Residencial Geminis
443,Upington,ZA,-28.4478,21.2561,56,River City Inn


### Add hotel name and country to the hover message for each city.

### Add hotel name and country to the hover message for each city, Plot the Map

In [8]:
#%%capture --no-display

# Configure the map plot
# Ensure hvplot is enabled for geographic plotting
hvplot.extension('bokeh')

# Configuring the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,  # Enable geographic coordinates
    size='Humidity',  # Use 'Humidity' for point sizes
    hover_cols=['City','Country', 'Hotel Name'],  # Data to show on hover
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=1000,
    frame_height=500,
    color='Humidity',  # Color points based on 'Humidity'
    cmap='Reds',  # Color map
    alpha=0.9,  # Transparency of the points
    xlim=(-110, 60),
    ylim=(-40, 25),
    xlabel='Longitude',  # Label for the X-axis
    ylabel='Latitude',   # Label for the Y-axis
    title=f'Potential Vacation Hotel Locations  -  {fetch_date}'  # Title of the plot
)
    
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)

### My Perfect Holiday Places

#### It looks like the perfect holiday destination in January 2024 for a beach loving Australian looking for almost cloudless skys is either Acapulco, Mexico, Mar del Plata, Argentina or Whakatane, New Zealand. 